#### passing parameters to the notebook (adding a widget)

In [0]:
dbutils.widgets.text("p_data_source", "")
v_data_source = dbutils.widgets.get("p_data_source")


#### invoke a notebook from another notebook using the %run command

In [0]:
%run "../Includes/configuration"


In [0]:
%run  "../Includes/common_functions"

#### Step1- Read the csv file using the spark dataframe reader

In [0]:
#circuit_df.printSchema()
#circuit_df.describe().show()

In [0]:
circuits_schema = StructType([
    StructField("circuitId", IntegerType(), False),
    StructField("circuitRef", StringType(), True),
    StructField("name", StringType(), True),
    StructField("location", StringType(), True),
    StructField("country", StringType(), True),
    StructField("lat", StringType(), True),
    StructField("lng", StringType(), True),
    StructField("alt", IntegerType(), True),
    StructField("url", StringType(), True)
])

In [0]:
circuit_df= spark.read\
    .options(header="true")\
    .schema(circuits_schema)\
    .csv(f"{raw_folder_path}/circuits.csv")

In [0]:
circuit_df= circuit_df.select(col("circuitId"), col("circuitRef"), col("name"), col("location"), col("country"), col("lat"), col("lng"))

In [0]:
circuits_renamed_df = add_ingestion_date(
    circuit_df
    .withColumnRenamed("circuitId", "circuit_id")
    .withColumnRenamed("circuitRef", "circuit_ref")
    .withColumnRenamed("lat", "latitude")
    .withColumnRenamed("lng", "longitude")
    .withColumnRenamed("alt", "altitude")
    .withColumn("data_source", lit(v_data_source))
)


In [0]:
circuits_renamed_df=circuits_renamed_df.drop("ingestion_date")

In [0]:
circuits_renamed_df.write.mode("overwrite").parquet(f"{processed_folder_path}/circuits")

In [0]:
dbutils.notebook.exit("Success")